In [2]:
import io
import arcpy
import pandas as pd
import requests
from arcgis.features import GeoAccessor, GeoSeriesAccessor, FeatureSet
from arcgis.widgets import MapView
from zipfile import ZipFile
import os
from osgeo import ogr
from arcgis.geometry import SpatialReference
import sys
from arcpy.sa import *
import arcpy

from arcpy import env

In [3]:
from arcgis import GIS
gis = GIS()

In [3]:
#access transit stop dataset via api
transitstops_link = r'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metc/trans_transit_stops/shp_trans_transit_stops.zip'

stops_response = requests.get(transitstops_link, allow_redirects=True)

In [4]:
#open a new file and use the dot interface. write to put data in the file
open("C:\\Users\\bende287\Downloads\\shp_trans_transit_stops.zip", 'wb').write(stops_response.content)

2411914

In [5]:
#unzip file
with ZipFile(r"C:\Users\bende287\Downloads\shp_trans_transit_stops.zip", 'r') as z:
    z.printdir()
    
    print('Extracting all files now..')
    z.extractall(r"C:\Users\bende287\Downloads\shp_trans_transit_stops")
    print('Done!')

File Name                                             Modified             Size
STOP_RouteListByStop.cpg                       2024-10-28 00:10:52            5
STOP_RouteListByStop.dbf                       2024-10-28 00:10:52       372546
STOP_RouteListByStop.dbf.xml                   2024-10-28 00:10:52         5286
TransitStops.cpg                               2024-10-28 00:10:54            5
TransitStops.dbf                               2024-10-28 00:10:54     23378132
TransitStops.prj                               2024-10-28 00:10:52          424
TransitStops.sbn                               2024-10-28 00:10:54       209300
TransitStops.sbx                               2024-10-28 00:10:54         5948
TransitStops.shp                               2024-10-28 00:10:54       607868
TransitStops.shp.xml                           2024-10-28 00:10:54        22609
TransitStops.shx                               2024-10-28 00:10:54       173748
metadata/metadata.html                  

In [6]:
# make it a spatially enabled df
df_stops = pd.DataFrame.spatial.from_featureclass(r"C:\Users\bende287\Downloads\shp_trans_transit_stops\TransitStops.shp")
#transform the projection 
df_stops.spatial.project(SpatialReference(4326))

df_stops

,FID,site_id,site_on,site_at,corn_desc,CornerNumb,descriptio,city,county,publiccomm,dt_zone,parkride,adaaccess,busstop_yn,node_id,gate,T_Code,UNIQUE_ID,board_flag,alight_fla,active_sta,itinerary_,IsSuspende,route_prov,SHAPE
0,0,34,Chicago Ave S,8th St S,Farside Southbound,5,Chicago Ave S & 8th St S,MINNEAPOLIS,Hennepin,,Y,N,Y,Y,8SCH,,,7A8DD863-8277-4ADC-A0F2-62D491721677,1,1,Active,Y,N,Metro Transit,"{""x"": -93.26242600042944, ""y"": 44.971094999627..."
1,1,36,Chicago Ave S,14th St E,Nearside Southbound,8,Chicago Ave S & 14th St E,MINNEAPOLIS,Hennepin,,N,N,Y,Y,14CH,,,6CBC5983-02F5-4CD8-9FB8-607496F5CF0D,0,0,Active,N,N,,"{""x"": -93.262632000329, ""y"": 44.96903899968294..."
2,2,37,Chicago Ave S,15th St E,Nearside Southbound,8,Chicago Ave S & 15th St E,MINNEAPOLIS,Hennepin,,N,N,Y,Y,,,,CBBD7A82-964E-412C-8F30-744EFBDCA66E,1,1,Active,Y,N,Metro Transit,"{""x"": -93.26263400006296, ""y"": 44.968187000189..."
3,3,39,Chicago Ave S,17th St E,Nearside Southbound,8,Chicago Ave S & 17th St E,MINNEAPOLIS,Hennepin,,N,N,Y,Y,,,,5CB2F36C-C1A1-4FDE-82FC-581DB8D9D6D5,1,1,Active,Y,N,Metro Transit,"{""x"": -93.26264099983634, ""y"": 44.966423000214..."
4,4,40,Chicago Ave S,18th St E,Farside Southbound,5,Chicago Ave S & 18th St E,MINNEAPOLIS,Hennepin,,N,N,Y,Y,,,,BDBC2BB1-1C23-46EC-B0A3-A502D94BC719,1,1,Active,Y,N,Metro Transit,"{""x"": -93.26263800011066, ""y"": 44.964945000415..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21701,21701,90371,Highway 10,Approaching Big Lake Station,Nearside Eastbound,6,Hwy 10 and County Rd 43 Control Pt,BIG LAKE,Sherburne,Control point - no stop,N,N,N,N,,,,D42980B4-4D80-4D87-B5FA-F1AFECB359DD,0,0,Active,Y,N,St Cloud Metro Bus,"{""x"": -93.73190600028269, ""y"": 45.332255999560..."
21702,21702,90372,171st Avenue Northwest,Approaching Elk River Station,Farside Eastbound,3,171st Ave NW and Hwy 10 Control Pt,ELK RIVER,Sherburne,Control point - no stop,N,N,N,N,,,,136A6CC5-488B-4E52-9963-E76A88A4FC9D,0,0,Active,N,N,,"{""x"": -93.54887900059705, ""y"": 45.281455999651..."
21703,21703,90373,Interstate 394,Approaching 12th Street - Downtown Minneapolis,Midblock Eastbound,10,I-394 to 12th St Exit Control Pt,MINNEAPOLIS,Hennepin,Control point - no stop,N,N,N,N,,,,CC52E289-5113-41EE-A5D3-20712CE58D45,0,0,Active,Y,N,Metro Transit,"{""x"": -93.28664300048878, ""y"": 44.974962000101..."
21704,21704,90374,Century Ave and I-94 Exit Control Point,Century Ave and I-94 Exit Control Point,Nearside Eastbound,6,Century Ave and I-94 Exit Control Pt,MAPLEWOOD,Ramsey,Control point - no stop,N,N,N,N,,,,5351E352-7480-4844-A09B-127AC4E83F09,0,0,Active,Y,N,Metro Transit,"{""x"": -92.98578399994945, ""y"": 44.947617000401..."


In [10]:
#access hennepin county boundary to clip other layers to 
hennepinboundary_link = r"https://gis.hennepin.us/arcgis/rest/services/HennepinData/BOUNDARIES/MapServer/1/query?where=1%3D1&f=json"

bdry_response = requests.get(hennepinboundary_link, stream=True).json()

In [11]:
type(bdry_response)

<class 'dict'>

In [15]:
#going from dictionary to feature set here
fs = FeatureSet.from_dict(bdry_response)

bdry_fs = fs.sdf
bdry_fs

#making sure to maintain the same projection throughout-- transforming to WGS 84
#arc_rest.spatial.project(SpatialReference(4326))
#arc_rest

,NAME_TXT,OBJECTID,SHAPE
0,HENNEPIN COUNTY,1,"{""rings"": [[[459137.16889999993, 5010412.97660..."


In [16]:
#bike and pedestrian system layer
bikeandpeds_link = r"https://gis.hennepin.us/arcgis/rest/services/HennepinData/TRANSPORTATION/MapServer/1/query?where=1%3D1&f=json"

bikeandpeds_response = requests.get(bikeandpeds_link, stream=True).json()

type(bikeandpeds_response)

<class 'dict'>

In [17]:
#going from dictionary to feature set here
fs2 = FeatureSet.from_dict(bikeandpeds_response)

bikeandpeds_fs = fs2.sdf
bikeandpeds_fs


,TRAILNAME,OBJECTID,SHAPE
0,<NA>,1,"{""paths"": [[[458854.48649999965, 4963932.01380..."
1,<NA>,2,"{""paths"": [[[458886.23950000014, 4963936.6953]..."
2,<NA>,3,"{""paths"": [[[458923.60780000035, 4963943.7355]..."
3,<NA>,4,"{""paths"": [[[459306.29590000026, 4964213.52899..."
4,<NA>,5,"{""paths"": [[[472044.51159999985, 4961658.24640..."
...,...,...,...
1995,<NA>,1996,"{""paths"": [[[457367.9462000001, 5000558.634299..."
1996,<NA>,1997,"{""paths"": [[[457358.34300000034, 5000067.23740..."
1997,<NA>,1998,"{""paths"": [[[457394.80229999963, 5002058.5033]..."
1998,<NA>,1999,"{""paths"": [[[457392.1763000004, 5001612.020400..."


In [27]:
#can't figure this one out-- there are 135 listings for this search as of 11/2. this is supposed to be the land for sale dataset. 
land_link = r'https://www.land.com/api/property/search/0/Hennepin-County-MN/all-land/no-house/'

land_response = requests.get(land_link, stream=True).content

#land_json = land_response.json()



ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [4]:
#Parcel dataset that will eventually be joined to the land for sale data.
parcel_link = r'https://gis.hennepin.us/arcgis/rest/services/HennepinData/LAND_PROPERTY/MapServer/1/query?where=1%3D1%20AND%201%3D1&f=json'

parcel_response = requests.get(parcel_link, stream=True).json()

type(parcel_response)

<class 'dict'>

In [5]:
#going from dictionary to feature set here
fs3 = FeatureSet.from_dict(parcel_response)

parcel_fs = fs3.sdf
parcel_fs


,SALE_CODE_NAME,OBJECTID,SHAPE
0,WARRANTY DEED,1,"{""rings"": [[[481758.6118999999, 4967622.9012],..."
1,,2,"{""rings"": [[[481827.7642000001, 4967621.761100..."
2,EXCLUDED FROM RATIO STUDIES,3,"{""rings"": [[[481827.44710000046, 4967582.15430..."
3,,4,"{""rings"": [[[481799.1160000013, 4967582.621400..."
4,,5,"{""rings"": [[[482035.27010000125, 4967618.3399]..."
...,...,...,...
1995,WARRANTY DEED,1996,"{""rings"": [[[482004.89219999965, 4976424.7015]..."
1996,OTHER – SEE CERTIFICATE OF REAL ESTATE VALUE (...,1997,"{""rings"": [[[482010.4545000009, 4976413.867699..."
1997,WARRANTY DEED,1998,"{""rings"": [[[481912.8088999996, 4976443.9617],..."
1998,,1999,"{""rings"": [[[481892.46570000146, 4976433.5561]..."


In [6]:
parks_URL = r'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metrogis/bdry_metro_colabtiv_parks/shp_bdry_metro_colabtiv_parks.zip'

parks_response = requests.get(parks_URL, allow_redirects=True)

In [7]:
#open a new file and use the dot interface. write to put data in the file
open("C:\\Users\\bende287\Downloads\\shp_bdry_metro_colabtiv_parks.zip", 'wb').write(parks_response.content)

5525043

In [8]:
#unzip file
with ZipFile(r"C:\Users\bende287\Downloads\shp_bdry_metro_colabtiv_parks.zip", 'r') as c:
    c.printdir()
    
    print('Extracting all files now..')
    c.extractall(r"C:\Users\bende287\Downloads\shp_bdry_metro_colabtiv_parks")
    print('Done!')

File Name                                             Modified             Size
MetroCollaborativeParks.cpg                    2024-09-23 17:59:50            5
MetroCollaborativeParks.dbf                    2024-09-23 17:59:50     12172519
MetroCollaborativeParks.prj                    2024-09-23 17:59:50          424
MetroCollaborativeParks.sbn                    2024-09-23 17:59:50        37308
MetroCollaborativeParks.sbx                    2024-09-23 17:59:50         2404
MetroCollaborativeParks.shp                    2024-09-23 17:59:50      8507476
MetroCollaborativeParks.shp.xml                2024-09-23 17:59:50        77943
MetroCollaborativeParks.shx                    2024-09-23 17:59:50        31020
metadata/metadata.html                         2024-08-06 16:11:02        13462
metadata/metadata.xml                          2024-08-06 16:11:02         6542
Extracting all files now..
Done!


In [9]:
# make it a spatially enabled df
df_parks = pd.DataFrame.spatial.from_featureclass(r"C:\Users\bende287\Downloads\shp_bdry_metro_colabtiv_parks\MetroCollaborativeParks.shp")
#transform the projection 
#df_parks.spatial.project(SpatialReference(4326))

df_parks

,FID,PARKID,PARKNAME,NAME_ALT,PARK_ADDR,CITY_MUNI,COUNTY,STATE,PARKSTATUS,WINTER_STS,FUNC_TYPE,LANDOWNER,OWNERTYPE,AGENCYNAME,AGENCYTYPE,YEAROPEN,LANDCOVER,WATERACRES,LAND_ACRES,PARK_ACRES,PLND_ACRES,TRL_MILES,ACQSOURCE,ACQMETHOD,ACQCOMMENT,DEVSOURCE,DEVCOMMENT,RESTRICTS,LWCFPROT,OTHERPROT,RESPRTCMNT,SERVICAREA,MGMT_PRIOR,VLTR_OPPS,SPEC_FEAT,PARK_HOURS,PKING_CAP,VISITORCTS,PASSHOLDER,PARK_EMAIL,PARK_PHONE,PARK_URL,MNTCEEMAIL,MNTCEPHONE,DATAAUTHOR,EDITED_BY,EDITED_DT,CREATED_BY,CREATED_DT,Shape_Leng,Shape_Area,SHAPE
0,0,1,Islands of Peace Park,,200 Charles St NE,Fridley,003,MN,Open,Open,Regional Park,Anoka County,03,Anoka County Parks,03,0,,0.0,0.0,70.035095,0,0.0,,,,,,,,,,,,,,6 AM to 30 minutes past sunset,0,0,0,,763-324-3300,http://www.ci.fridley.mn.us/DocumentCenter/Vie...,,763-571-3450,Anoka County,HOEKENJM,2020-02-05,HOEKENJM,2020-02-05,4937.894919,283421.975373,"{""rings"": [[[477871.7566999998, 4991062.148499..."
1,1,2,Coon Lake County Park,,5450 197th Ave NE,Columbus,003,MN,Open,Open,County Park,Anoka County,03,Anoka County Parks,03,0,,0.0,0.0,111.440554,0,0.0,,,,,,,,,,,,,,6 AM to 30 minutes past sunset,0,0,0,,763-324-3300,https://www.anokacounty.us/722/Coon-Lake-Count...,,763-324-3300,Anoka County,HOEKENJM,2020-02-05,HOEKENJM,2020-02-05,3681.878922,450983.919852,"{""rings"": [[[490172.79389999993, 5019062.42809..."
2,2,3,Manomin Park,,6666 East River Rd,Fridley,003,MN,Open,Open,Regional Park,Anoka County,03,Anoka County Parks,03,0,,0.0,0.0,13.766334,0,0.0,,,,,,,,,,,,,,6 AM to 30 minutes past sunset,0,0,0,,763-324-3300,http://www.ci.fridley.mn.us/DocumentCenter/Vie...,,763-571-3450,Anoka County,HOEKENJM,2020-02-05,HOEKENJM,2020-02-05,1204.812861,55710.376642,"{""rings"": [[[478289.28330000024, 4993075.01789..."
3,3,4,Riverfront Regional Park,,5100 East River Rd,Fridley,003,MN,Open,Open,Regional Park,Anoka County,03,Anoka County Parks,03,0,,0.0,0.0,66.14842,0,0.0,,,,,,,,,,,,,,6 AM to 30 minutes past sunset,0,0,0,,763-324-3300,http://www.ci.fridley.mn.us/DocumentCenter/Vie...,,763-571-3450,Anoka County,HOEKENJM,2020-02-05,HOEKENJM,2020-02-05,4581.75324,267693.160293,"{""rings"": [[[477894.19570000004, 4990291.52270..."
4,4,5,Lake George Regional Park,,2859 Lake George Dr NW,Oak Grove,003,MN,Open,Open,Regional Park,Anoka County,03,Anoka County Parks,03,0,,0.0,0.0,257.882298,0,0.0,,,,,,,,,,,,,,6 AM to 30 minutes past sunset,0,0,0,,763-324-3300,http://www.ci.oak-grove.mn.us/vertical/sites/%...,,763-404-7000,Anoka County,HOEKENJM,2020-02-05,HOEKENJM,2020-02-05,5218.489393,1043612.6344,"{""rings"": [[[474097.65950000007, 5023290.9901]..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3860,3860,163-3821,Lake Elmo Park Reserve,,1515 KEATS AVE N,Lake Elmo,163,MN,Open Fee,Open,,Washington County,03,Washington County,03,0,,0.0,0.0,2177.03488,0,0.0,,,,,,,,,,,,,Swim Pond,6 A.M. - 10 P.M.,0,0,0,parks@co.washington.mn.us,651-430-8370,https://www.co.washington.mn.us/502/Lake-Elmo-...,,,Washington County,HOEKENJM,2020-02-05,HOEKENJM,2020-02-05,381.50367,8085.588943,"{""rings"": [[[509222.81599999964, 4979310.0371]..."
3861,3861,163-3822,Oak Park Crossing Park,,N/A,Oak Park Heights,163,MN,Open,Unknown,,XCEL Energy,99,Oak Park Heights,05,2015,,0.0,0.0,35.0,0,0.0,,,,,,,,,,,,,,,0,0,0,,,,,,Oak Park Heights,HOEKENJM,2020-02-05,HOEKENJM,2020-02-05,1855.172978,138922.342238,"{""rings"": [[[516126.4826999996, 4986830.0363],..."
3862,3862,163-3824,Autumn Hills Park,,N/A,Oak Park Heights,163,MN,Open,Unknown,,VALLEY SENIOR SERV ALLIANCE,99,Oak Park Heights,05,2007,,0.0,0.0,0.0,0,0.0,,,,,,,,,,,,,,,0,0,0,,,,,,Oak Park Heights,HOEKENJM,2020-02-05,HOEKENJM,2020-02-05,1064.730402,73824.970108,"{""rings"": [[[513179.2923999997, 4986383.2434],..."
3863,3863,163-3825,Linear Park,,N/A,Oak Park Heights,163,MN,Open,Unknown,,Oak Park Heights,05,Oak Park Heights,05,0,,0.0,0.0,0.0,0,0.0,,,,,,,,,,,,,,

In [ ]:
#Still need to bring in demographic dataset

In [ ]:
#Next cells are going to clip my layers 

In [ ]:
arcpy.analysis.Clip(
    in_features="TransitStops",
    clip_features="Hennepin_County_Boundary",
    out_feature_class=r"C:\Users\bende287\Documents\ArcGIS\Projects\GIS5571FinalProj\GIS5571FinalProj.gdb\TransitStops_Clip",
    cluster_tolerance=None
)